In [1]:
# load the necessary libraries for the project
import pandas as pd
import jieba
from collections import Counter
import string
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import spacy
import jieba.posseg as pseg
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora
from gensim.models import LdaMulticore
from gensim.models import LdaMulticore, CoherenceModel
from gensim.models import TfidfModel
from gensim.matutils import sparse2full
from itertools import islice
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from transformers.pipelines import pipeline
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN
from umap import UMAP
from fastopic import FASTopic
import topmost
from topmost.data import RawDataset
from topmost.preprocessing import Preprocessing
# from topicgpt.TopicGPT import TopicGPT
import plotly.io as pio
pio.renderers.default = 'notebook'

In [3]:
# load the csv file data
speech_df = pd.read_csv('governance_of_china_all_volumes_with_formatted_date.csv')

# sort the data by date
speech_df = speech_df.sort_values('year')

# check the first few rows of the data
speech_df.head()

,title,date,text,references,formatted_date,year
0,就是我们的奋斗目标 * (Speech 1),（2012年11月15日）,* 这是习近平在十八届中央政治局常委同中外记者见面时讲话的主要部分。\n\n记者朋友们对十八...,NaN,2012-11-15,2012
38,不断推向前进 * (Speech 39),（2012年11月16日）,* 这是习近平在中央军委扩大会议上的讲话要点。\n\n军委班子和军队高级干部在领导国防和军队...,注 释\n\n〔1〕 胡主席，即胡锦涛。\n〔2〕 毛主席，即毛泽东。\n〔3〕 邓主席，即...,2012-11-16,2012
20,施行30周年大会上的讲话 (Speech 21),（2012年12月4日）,同志们，朋友们：\n年12月4日，五届全国人大五次会议通过了《中华人民共和国 \n宪法》。我...,注 释\n\n〔1〕 新民主主义革命，是由无产阶级领导的、革命性质仍然是反对帝国主义反对封建...,2012-12-04,2012
10,改革开放只有进行时没有完成时 * (Speech 11),（2012年12月31日）,* 这是习近平在主持十八届中央政治局第二次集体学习时的讲话要点。\n\n改革开放是一项长期的...,NaN,2012-12-31,2012
15,实实在在和没有水分的增长 * (Speech 16),（2012年11月30日）,* 这是习近平在中共中央召开的党外人士座谈会上的讲话要点。\n\n今年以来，面对国际经济的复...,NaN,2012-11-30,2012


In [4]:
# prepare the text for topic modeling
corpus = speech_df['text'] 
# remove the empty documents
corpus = [doc for doc in corpus if doc != ""]
# Remove newlines and extra whitespaces within sentences in each document
corpus = [re.sub(r'\s+', '', doc).strip() for doc in corpus]
# Remove the first sentence from each document
corpus = [re.sub(r'^[^。]*。', '', doc) for doc in corpus]
# remove the intext citations 
corpus = [re.sub(r'〔\d+〕', '', doc) for doc in corpus]

corpus[:10]

['记者朋友们对十八大作了大量报道，向世界传递了许多“中国声音”。我代表大会秘书处向大家表示衷心的感谢。刚才，我们召开了中国共产党第十八届中央委员会第一次全体会议，选举产生了新一届中央领导机构，选举我为中央委员会总书记。我代表新一届中央领导机构成员感谢全党同志的信任，定当不负重托，不辱使命。全党同志的重托，全国各族人民的期望，是对我们做好工作的巨大鼓舞，也是我们肩上的重大责任。这个重大责任，就是对民族的责任。我们的民族是伟大的民族。在五千多年的文明发展历程中，中华民族为人类文明进步作出了不可磨灭的贡献。近代以后，我们的民族历经磨难，中华民族到了最危险的时候。自那时以来，为了实现中华民族伟大复兴，无数仁人志士奋起抗争，但一次又一次地失败了。中国共产党成立后，团结带领人民前仆后继、顽强奋斗，把贫穷落后的旧中国变成日益走向繁荣富强的新中国，中华民族伟大复兴展现出前所未有的光明前景。我们的责任，就是要团结带领全党全国各族人民，接过历史的接力棒，继续为实现中华民族伟大复兴而努力奋斗，使中华民族更加坚强有力地自立于世界民族之林，为人类作出新的更大的贡献。这个重大责任，就是对人民的责任。我们的人民是伟大的人民。在漫长的历史进程中，中国人民依靠自己的勤劳、勇敢、智慧，开创了各民族和睦共处的美好家园，培育了历久弥新的优秀文化。我们的人民热爱生活，期盼有更好的教育、更稳定的工作、更满意的收入、更可靠的社会保障、更高水平的医疗卫生服务、更舒适的居住条件、更优美的环境，期盼孩子们能成长得更好、工作得更好、生活得更好。人民对美好生活的向往，就是我们的奋斗目标。人世间的一切幸福都需要靠辛勤的劳动来创造。我们的责任，就是要团结带领全党全国各族人民，继续解放思想，坚持改革开放，不断解放和发展社会生产力，努力解决群众的生产生活困难，坚定不移走共同富裕的道路。这个重大责任，就是对党的责任。我们的党是全心全意为人民服务的政党。党领导人民已经取得举世瞩目的成就，我们完全有理由因此而自豪，但我们自豪而不自满，决不会躺在过去的功劳簿上。新形势下，我们党面临着许多严峻挑战，党内存在着许多亟待解决的问题。尤其是一些党员干部中发生的贪污腐败、脱离群众、形式主义、官僚主义等问题，必须下大气力解决。全党必须警醒起来。打铁还需自身硬。我们的责任，就是同全党同志一道，坚持党要管党、从严治党，切实解决自身存在的突出问题，切实

In [5]:
############## load the tokenizer for Chinese text spacy version ##############

# Load the Chinese spaCy model
zh_core = spacy.load("zh_core_web_sm")

# Paths to your custom stop words and vocabulary files
stopwords_path = "chinese_stopwords.txt"
custom_words_path = "custom_word_list.txt"

# Function to load stop words from a file
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        stopwords = set(file.read().splitlines())
    return stopwords

# Function to load custom vocabulary from a file
def load_custom_vocab(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        custom_vocab = set(file.read().splitlines())
    return custom_vocab

# Load custom stop words and custom vocabulary
custom_stopwords = load_stopwords(stopwords_path)
custom_vocab = load_custom_vocab(custom_words_path)

# Add custom stop words to spaCy's stop words set
for word in custom_stopwords:
    zh_core.vocab[word].is_stop = True

# Add custom words to spaCy's vocabulary (if they aren't already recognized)
for word in custom_vocab:
    _ = zh_core.vocab[word]  # This ensures the word is added to the vocabulary

# Define a custom tokenizer function
def zh_tokenizer(text):
    # Tokenize the text and filter out stop words and spaces
    tokens = [token.text for token in zh_core(text) if not token.is_stop and not token.is_space]
    return tokens

# Example usage
sample_text = "第三，必须锐意改革、大胆创新。我们面临的困难和问题，确实同国际金融危机这一外因的影响有直接关系，但内因是起决定性作用的，内因就是我们正面对着深刻的供给侧、结构性、体制性矛盾"
tokens = zh_tokenizer(sample_text)

print(tokens)


['第三', '锐意', '改革', '大胆', '创新', '面临', '困难', '确实', '国际', '金融', '危机', '外因', '影响', '关系', '内因', '决定性', '作用', '内因', '正', '面对', '深刻', '供给侧', '结构性', '体制性', '矛盾']


In [6]:
# load customized words for jieba
stopwords_path = "chinese_stopwords.txt"
custom_words_path = "custom_word_list.txt"

# Clear any existing dictionary settings
jieba.initialize()

# Load the custom dictionary and print confirmation
print(f"Loading custom dictionary from: {custom_words_path}")
jieba.load_userdict(custom_words_path)

# Verify if words are in the dictionary
print("Checking if '供给侧' is in dictionary:", '供给侧' in jieba.dt.FREQ)

def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        stopwords = set(file.read().splitlines())
    return stopwords

stopwords = load_stopwords(stopwords_path)

def jieba_tokenizer(text):
    words = jieba.lcut(text, cut_all=False)
    filtered_tokens = []
    for word in words:
        if (word not in stopwords and 
            word.strip()): 
            filtered_tokens.append(word)
    return filtered_tokens

# Test with a specific example
test_text = "第三，必须锐意改革、大胆创新。我们面临的困难和问题，确实同国际金融危机这一外因的影响有直接关系，但内因是起决定性作用的，内因就是我们正面对着深刻的供给侧、结构性、体制性矛盾"
tokens = jieba_tokenizer(test_text)
print(tokens)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\brobo\AppData\Local\Temp\jieba.cache
Loading model cost 0.405 seconds.
Prefix dict has been built successfully.


Loading custom dictionary from: custom_word_list.txt
Checking if '供给侧' is in dictionary: True
['第三', '锐意改革', '大胆', '创新', '面临', '困难', '问题', '确实', '国际', '金融危机', '这一', '外因', '影响', '直接', '关系', '内因', '决定性', '作用', '内因', '正面', '深刻', '供给侧', '结构性', '体制性', '矛盾']


In [7]:
# Train the FASTopic model spacy version 
import random
seed = 42
random.seed(seed)
np.random.seed(seed)

preprocessing = Preprocessing(vocab_size=10000, tokenizer=zh_tokenizer)

model = FASTopic(50, preprocessing, doc_embed_model='paraphrase-multilingual-MiniLM-L12-v2', verbose=True)
top_words, train_theta = model.fit_transform(corpus)


2024-11-30 20:29:42,945 - FASTopic - use device: cuda
parsing texts: 100%|██████████| 337/337 [00:23<00:00, 14.33it/s]
2024-11-30 20:30:54,330 - TopMost - Real vocab size: 10000
2024-11-30 20:30:54,333 - TopMost - Real training size: 337 	 avg length: 731.766
Training FASTopic: 100%|██████████| 200/200 [01:12<00:00,  2.75it/s]

Topic 0: 停歇 大敌 纪法 通用 民族主义 大汉族主义 兴边 侵吞 抓纲带目 党学 内设 堤坝 厚植党 海晏河清 痛恨
Topic 1: 乡村 大循环 循环 农民 农业 收入 振兴 富裕 农村 城乡 长三角 城市 城市群 国内 常住
Topic 2: 两岸 台湾 同胞 内地 一国两制 港澳 香港 澳门 同胞们 怀抱 心结 晚年 一中一台 基本法 行政区
Topic 3: 人权 政协 商量 协商 铸牢 民主 人权观 人选 共同性 追溯 刻板 广开言路 生存权 发展权 嵌入
Topic 4: 核 进口 贸易港 联合国 总理 自由 日内瓦 贸易 外资 浦东 尊敬 先生们 女士们 协定 放宽
Topic 5: 阶级 工人 劳模 近平 雄关 号 漫道 如铁 屈服 中坚 飞机 知识型 骐骥 羽 苦苦
Topic 6: 产销 保住 借贷 增产 玉米 一分为二 农惠 攻关期 稀缺 矫正 二元 岛 大进 纲领性 上涨
Topic 7: 房住 城里 研协 后富 依靠党 于民 聚焦党 不纯 做细 更实 先试 行则 确保党 常新 错综复杂
Topic 8: 美 奥巴马 论调 哈萨克 波罗的海 通话 阿曼盖尔德 两千多 19万 事在人为 总人口 美利坚 积微成著 举目无亲 2万多
Topic 9: 全会 三中全会 机构 职能 起草组 配置 改革 政治局 征求 事权 十八 深化 意见 决定性 体制
Topic 10: 特色 意识 建设 新时代 统一 文化 团结 工作 政治 中华 复兴 坚强 实践 能力 增强
Topic 11: 中欧 巴基斯坦 目自张 政治关 难于法 站在党 养以成 多层 学习党 聚精会 知荣辱 抗日战争 党治国 凡属 脑科学
Topic 12: 16亿 贵阳 风电 共建网 平年 之初 打动人 几千年 总钥匙 铁一般 拘泥于 头上 管党 破坏党 百年大党
Topic 13: 胡主席 筹划 枪杆子 邓主席 宣导 人手 国共 立军 江 军令 守纪律 讲格调 大进步 事多 中拉
Topic 14: 宪法 政法 立法 法治 司法 法律 守法 治国 人大 依法 执法 行政 法规 常委会 行使
Topic 15: 群团 先进性 初心 纯洁性 动摇党 忘 逆境 终同 党始 顺境 自我 弱化党 危险 检视 使命
Topic 16: 卫生 实物 稳岗 地防 返还 防病 中西药 诊断 确诊

In [8]:
model.get_topic(topic_idx=38)

(('科技', 0.0071234484),
 ('数字', 0.005212542),
 ('技术', 0.005172282),
 ('科研', 0.0050146207),
 ('人才', 0.004853417))

In [9]:
fig = model.visualize_topic(top_n=10)
fig.show()

In [10]:
# Train the FASTopic model jieba version 
preprocessing = Preprocessing(vocab_size=10000, tokenizer= jieba_tokenizer)

model = FASTopic(50, preprocessing, doc_embed_model='paraphrase-multilingual-MiniLM-L12-v2', verbose=True)
top_words, train_theta = model.fit_transform(corpus)

2024-11-30 20:32:12,258 - FASTopic - use device: cuda
parsing texts: 100%|██████████| 337/337 [00:01<00:00, 200.91it/s]
2024-11-30 20:32:18,351 - TopMost - Real vocab size: 10000
2024-11-30 20:32:18,353 - TopMost - Real training size: 337 	 avg length: 787.172
Training FASTopic: 100%|██████████| 200/200 [00:52<00:00,  3.79it/s]

Topic 0: 人权 宪法 人大 全面依法治国 人民代表大会 政治制度 当家作主 民主专政 民主选举 保障人权 国家机关 司法 全过程人民民主 国体 法律
Topic 1: 疫苗 多边主义 抗疫 脱钩 联合国 发展中国家 议程 倡议 复苏 全球治理 赤字 对抗 人类 秩序 赢
Topic 2: 外交活动 完美无缺 自然而然 一尊 要谋 欠缺 速滑 组长 一羽 非一足 之动 之速 之轻 诞辰 高远
Topic 3: 乡村 农民 农村 循环 振兴 国内大循环 农业 城乡融合发展 乡风 兴旺 扩大内需 三农 收入分配差距 生活富裕 先手棋
Topic 4: 地点 地带 贫困 扶贫 受灾 脱贫 党校 讲话 去产能 发放 不久 县委 脱贫攻坚 贫困家庭 一篇
Topic 5: 生物 生态环境 环境保护 防控 防治 污染 生态文明建设 生态 卫生 绿化 保护 医务人员 疫情 公共卫生 修复
Topic 6: 我军 全军 军事 治军 强军 军队 打仗 战斗力 国防 军委 中央军委 官兵 后勤 军事战略 军费
Topic 7: 人民政协 协商民主 1992 稿 众人 党和国家 治理 定型 经济体制 制度 商量 协商 全面深化改革 建议 成熟
Topic 8: 俄 两 总统 两国人民 俄罗斯 两国关系 中俄关系 中美关系 普京 之桥 双边关系 俄方 热情好客 通话 时隔
Topic 9: 沉甸甸 第十六 犯下 稀缺资源 趋缓 挑起 水质 受到冲击 1.2 极地 下调 食物 慢性病 逐项 严重威胁
Topic 10: 巴基斯坦 繁荣昌盛 路遥知马力 东盟自由贸易区 日久见人心 得天独厚 作出努力 申请 拉美 印尼 避免出现 腾飞 多变 更多正 俱乐部
Topic 11: 东盟 丝绸之路 上海 东盟国家 亚太 成员国 互联互通 哈萨克斯坦 主席国 三股 观察员 阿富汗 伙伴 亚信 中方
Topic 12: 中欧 沿途 家乡 火 名录 独具 一无是处 习俗 羹 大夫 口碑 活跃期 宽阔 西域 友好往来
Topic 13: 长征 邓小平 伟大 中国共产党 真理 胜利 永远 人民军队 同志 理想 精神 奋斗 征程 创造 前进
Topic 14: 之祸 海峡 深切体会 伤痛 心之痛 晚年 第十八届 身居 职位 逆转 粗暴 功劳簿 躺 再长 流淌
Topic 15: 资本 资源配置 监管 长三角

In [11]:
model.get_topic(topic_idx=8)

(('俄', 0.0075599505),
 ('两', 0.006829163),
 ('总统', 0.0055864933),
 ('两国人民', 0.0050525856),
 ('俄罗斯', 0.0049479497))

In [12]:
fig = model.visualize_topic(top_n=10)
fig.show()

In [13]:
######### BERTopic #########
# vectorize the text data using the tokenizer
spacy_vectorizer = CountVectorizer(tokenizer=zh_tokenizer)
jieba_vectorizer = CountVectorizer(tokenizer=jieba_tokenizer)

### KeyBERTInspired
from bertopic.representation import KeyBERTInspired
keybert_model = KeyBERTInspired()

### MMR
from bertopic.representation import MaximalMarginalRelevance
mmr_model = MaximalMarginalRelevance(diversity=0.3)

### Zero-Shot Classification
from bertopic.representation import ZeroShotClassification
candidate_topics = ['中国梦', '四个全面', '反腐倡廉', '扶贫攻坚', '一带一路', '供给侧结构性改革','共同富裕', '生态文明', '人类命运共同体']
zero_shot_model = ZeroShotClassification(candidate_topics, model = "morit/chinese_xlm_xnli")

# Create your representation model
representation_model = {
    "KeyBERT": keybert_model,
    "MMR": mmr_model,
    "Zero-Shot": zero_shot_model
}

# Create your embedding model
sentence_model = SentenceTransformer('lier007/xiaobu-embedding-v2')

In [20]:
from hdbscan import HDBSCAN
from umap import UMAP
from sklearn.cluster import KMeans

hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

kmeans_model = KMeans(n_clusters=15)

topic_model_spacy = BERTopic(
    embedding_model=sentence_model,
    language="multilingual",
    calculate_probabilities=True,
    verbose=True,
    vectorizer_model=spacy_vectorizer,
    representation_model=representation_model,
    umap_model=umap_model,
    min_topic_size=10,
    )

topics, probs = topic_model_spacy.fit_transform(corpus)

2024-11-30 20:39:06,245 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

2024-11-30 20:39:20,196 - BERTopic - Embedding - Completed ✓
2024-11-30 20:39:20,196 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-30 20:39:21,190 - BERTopic - Dimensionality - Completed ✓
2024-11-30 20:39:21,191 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-30 20:39:21,200 - BERTopic - Cluster - Completed ✓
2024-11-30 20:39:21,202 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-30 20:41:00,680 - BERTopic - Representation - Completed ✓


In [21]:
freq = topic_model_spacy.get_topic_info(); freq.head(20)

,Topic,Count,Name,Representation,KeyBERT,MMR,Zero-Shot,Representative_Docs
0,-1,28,-1_发展_工作_国际_网络,"[发展, 工作, 国际, 网络, 合作, 外交, 社会, 世界, 全球, 疫情]","[形势, 团结, 大国, 重视, 统一, 党, 对外, 联合国, 外交, 总体]","[发展, 外交, 疫情, 我国, 治理, 媒体, 维护, 党, 联合国, 对外]","[发展, 工作, 国际, 网络, 合作, 外交, 社会, 世界, 全球, 疫情, 新, 我国...",[今年是中国共产党成立100周年。今年也是中华人民共和国恢复在联合国合法席位周年，中国将隆重...
1,0,211,0_发展_党_社会_建设,"[发展, 党, 社会, 建设, 改革, 工作, 新, 制度, 经济, 政治]","[改革, 党, 依法, 民主, 深化, 体制, 坚定, 推进, 统一, 提出]","[发展, 党, 改革, 新, 制度, 我国, 体系, 干部, 治理, 提高]","[发展, 党, 社会, 建设, 改革, 工作, 新, 制度, 经济, 政治, 我国, 领导,...",[党的十八大报告勾画了在新的历史条件下全面建成小康社会、加快推进社会主义现代化、夺取中国特色...
2,1,39,1_合作_发展_世界_各国,"[合作, 发展, 世界, 各国, 非洲, 经济, 一路, 一带, 国际, 中非]","[共建, 共赢, 共同体, 全球化, 多边主义, 携手, 维护, 金砖, 中方, 倡议]","[发展, 一带, 中非, 共建, 和平, 文明, 推动, 东盟, 中方, 发展中国家]","[合作, 发展, 世界, 各国, 非洲, 经济, 一路, 一带, 国际, 中非, 人类, 共...",[共建“一带一路”倡议，目的是聚焦互联互通．深化务实合作，携手应对人类面临的各种风险挑战，实...
3,2,26,2_发展_经济_合作_世界,"[发展, 经济, 合作, 世界, 国际, 各国, 亚洲, 开放, 贸易, 文明]","[尊敬, 朋友们, 维护, 共赢, 联合国, 中方, 各方, 全球化, 尊重, 友好]","[发展, 合作, 各国, 开放, 文明, 推动, 支持, 女士们, 朋友们, 一带]","[发展, 经济, 合作, 世界, 国际, 各国, 亚洲, 开放, 贸易, 文明, 增长, 和...",[尊敬的联合国大会主席汤姆森先生，尊敬的联合国秘书长古特雷斯先生，尊敬的联合国日内瓦总部总干...
4,3,21,3_两岸_同胞_民族_发展,"[两岸, 同胞, 民族, 发展, 中华, 和平, 澳门, 香港, 台湾, 关系]","[团结, 两岸, 统一, 一国两制, 同志们, 共产党, 维护, 爱国, 党, 同胞]","[两岸, 中华, 一国两制, 复兴, 祖国, 统一, 特别, 维护, 党, 团结]","[两岸, 同胞, 民族, 发展, 中华, 和平, 澳门, 香港, 台湾, 关系, 世界, 一...",[过去5年，我们两党、两岸双方和两岸同胞共同努力，开辟了两岸关系和平发展的正确道路，推动两岸...
5,4,12,4_军队_军事_强军_国防,"[军队, 军事, 强军, 国防, 建设, 工作, 改革, 军民, 政治, 战略]","[军委, 我军, 强军, 国防, 方针, 治军, 全军, 体制, 部队, 打胜仗]","[强军, 改革, 党, 全军, 军委, 贯彻, 我军, 战斗力, 治军, 澳门]","[军队, 军事, 强军, 国防, 建设, 工作, 改革, 军民, 政治, 战略, 人才, 党...",[军委班子和军队高级干部在领导国防和军队建设中肩负着重大历史责任。我们要始终保持清醒头脑，倍...


In [23]:
fig = topic_model_spacy.visualize_topics(); fig

In [17]:
topic_model_jieba = BERTopic(
    embedding_model=sentence_model,
    language="multilingual",
    calculate_probabilities=True,
    verbose=True,
    vectorizer_model=jieba_vectorizer,
    representation_model=representation_model
    )

topics, probs = topic_model_jieba.fit_transform(corpus)

2024-11-30 20:36:00,684 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/11 [00:00<?, ?it/s]

2024-11-30 20:36:14,790 - BERTopic - Embedding - Completed ✓
2024-11-30 20:36:14,791 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-11-30 20:36:16,474 - BERTopic - Dimensionality - Completed ✓
2024-11-30 20:36:16,475 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-11-30 20:36:16,487 - BERTopic - Cluster - Completed ✓
2024-11-30 20:36:16,490 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-11-30 20:36:24,495 - BERTopic - Representation - Completed ✓


In [18]:
freq = topic_model_jieba.get_topic_info(); freq.head(20)

,Topic,Count,Name,Representation,KeyBERT,MMR,Zero-Shot,Representative_Docs
0,-1,19,-1_发展_人民_外交_工作,"[发展, 人民, 外交, 工作, 世界, 坚持, 国际, 媒体, 疫情, 我国]","[党中央, 中国共产党, 奋斗目标, 加强, 和平发展, 重要, 重大, 维护, 党, 大局]","[外交, 媒体, 疫情, 我国, 对外, 联合国, 新, 治理, 加强, 维护]","[发展, 人民, 外交, 工作, 世界, 坚持, 国际, 媒体, 疫情, 我国, 安全, 合...",[今年是中国共产党成立100周年。今年也是中华人民共和国恢复在联合国合法席位周年，中国将隆重...
1,0,218,0_发展_党_人民_坚持,"[发展, 党, 人民, 坚持, 问题, 工作, 建设, 我国, 制度, 新]","[中国特色社会主义, 党中央, 坚决, 改革, 改革开放, 党, 社会主义, 加强, 重要,...","[党, 我国, 加强, 改革, 社会主义, 干部, 中国特色社会主义, 重大, 法治, 治理]","[发展, 党, 人民, 坚持, 问题, 工作, 建设, 我国, 制度, 新, 社会, 政治,...",[党的十八大报告勾画了在新的历史条件下全面建成小康社会、加快推进社会主义现代化、夺取中国特色...
2,1,41,1_合作_发展_各国_世界,"[合作, 发展, 各国, 世界, 非洲, 一带一路, 国际, 经济, 共同, 中非]","[一带一路, 共同发展, 共建, 金砖, 共赢, 全球治理, 共同体, 全球化, 面临, 发展]","[一带一路, 中非, 共建, 文明, 加强, 应该, 中方, 发展中国家, 共赢, 东盟]","[合作, 发展, 各国, 世界, 非洲, 一带一路, 国际, 经济, 共同, 中非, 共建,...",[共建“一带一路”倡议，目的是聚焦互联互通．深化务实合作，携手应对人类面临的各种风险挑战，实...
3,2,27,2_发展_经济_合作_世界,"[发展, 经济, 合作, 世界, 各国, 国际, 亚洲, 安全, 开放, 应该]","[尊敬, 共同发展, 联合国, 十国集团, 维护, 共赢, 应该, 全球化, 中方, 各方]","[发展, 合作, 各国, 开放, 应该, 文明, 共同, 女士们, 促进, 一带一路]","[发展, 经济, 合作, 世界, 各国, 国际, 亚洲, 安全, 开放, 应该, 增长, 推...",[尊敬的联合国大会主席汤姆森先生，尊敬的联合国秘书长古特雷斯先生，尊敬的联合国日内瓦总部总干...
4,3,20,3_两岸_人民_发展_澳门,"[两岸, 人民, 发展, 澳门, 同胞, 两岸关系, 香港, 实现, 一国两制, 坚持]","[中华民族伟大复兴, 两岸关系, 团结, 台湾同胞, 党中央, 坚决, 港澳同胞, 统一, ...","[澳门, 两岸关系, 一国两制, 中华民族, 和平发展, 共同, 中华民族伟大复兴, 中国共...","[两岸, 人民, 发展, 澳门, 同胞, 两岸关系, 香港, 实现, 一国两制, 坚持, 中...",[我将忠实履行宪法赋予的职责，忠于祖国，忠于人民，恪尽职守，夙夜在公，为民服务，为国尽力，自...
5,4,12,4_军队_军事_强军_国防,"[军队, 军事, 强军, 国防, 建设, 工作, 改革, 我军, 政治, 党]","[中央军委, 军委, 国防建设, 我军, 强军, 人民军队, 战略方针, 国防, 军民融合,...","[强军, 我军, 党, 全军, 贯彻, 治军, 新时代, 人民军队, 加强, 中央军委]","[军队, 军事, 强军, 国防, 建设, 工作, 改革, 我军, 政治, 党, 全军, 发展...",[军事政策制度调节军事关系、规范军事实践、保障军事发展，军事政策制度改革对实现党在新时代的强...


In [19]:
fig = topic_model_jieba.visualize_topics(); fig